# 네이버 뉴스기사 크롤링 분류 프로젝트

목적: 네이버 뉴스 기사를 크롤링해서 카테고리 분류

- 카테고리 번호
    - 100: 정치
    - 101: 경제
    - 102: 사회
    - 103: 생활/문화
    - 104: 세계
    - 105: IT/과학


평기기준:
1. 한국어 전치 과정이 적젏하였는가?
    - 형태소 분석기 선택과 불용어 제거가 체계적으로 진행됨
2. 텍스트 데이터 수집이 분량과 다양성 측면에서 저절했는가?
    - 일자와 분량에서 텍스트 데이터 다양성 향상을 위한 노력이 확인됨
3. 분류모델의 test accuracy가 기준 이상 높게 나왔는가?
    - f-1 score 기준 83% 이상의 정확도가 확인됨


## 크롤러 만들기

### BeautifulSoup

In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article

#### make_urllist

URL 리스트를 리턴하는 함수 </br>
원하는 페이지수, 카테고리 번호, 날짜를 입력값으로 받는다.

In [3]:
def make_urllist(page_num, code, date): 
  urllist= []
  for i in range(1, page_num + 1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}
    # lang="ko" data-useragent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    news = requests.get(url, headers=headers)

    # BeautifulSoup의 인스턴스 생성(파서는 html.parser를 사용)
    soup = BeautifulSoup(news.content, 'html.parser')

    # CASE 1
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    
    # CASE 2
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
        
    # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만 가져옴
    for line in news_list:
        urllist.append(line.a.get('href'))
  return urllist

In [4]:
url_list = make_urllist(2, 105, 20231128)
print(f'뉴스 기사 개수 = {len(url_list)}')

뉴스 기사 개수 = 40


In [5]:
url_list[:5]

['https://n.news.naver.com/mnews/article/009/0005222155?sid=105',
 'https://n.news.naver.com/mnews/article/020/0003533945?sid=105',
 'https://n.news.naver.com/mnews/article/092/0002313077?sid=105',
 'https://n.news.naver.com/mnews/article/008/0004967473?sid=105',
 'https://n.news.naver.com/mnews/article/056/0011611479?sid=105']

### newspaper3k

#### make_data

URL 리스트와 해당 URL이 어떤 카테고리인지 코드를 알려주면 이를 통해 데이터프레임 생성하는 함수 </br>
URL_list와, 카테고리를 입력 받는다.

In [9]:
idx2word = {'100' : '정치', '101' : '경제', '102' : '사회', '103' : '생활/문화', '104' : '세계','105' : 'IT/과학'}

def make_data(urllist, code):
    text_list = []
    for url in urllist:
        article = Article(url, language='ko')
        article.download()
        article.parse()
        text_list.append(article.title)

    #- 데이터프레임의 'news' 키 아래 파싱한 텍스트를 밸류로 붙여줍니다.
    df = pd.DataFrame({'news': text_list})

    #- 데이터프레임의 'code' 키 아래 한글 카테고리명을 붙여줍니다.
    df['code'] = idx2word[str(code)]
    return df

In [11]:
data = make_data(url_list, 105)
data[:10]

,news,code
0,우리 할머니 인슐린 주사 그만 맞아도 될까…‘당뇨 완치’ 길 열리나,IT/과학
1,"강스템바이오텍, ‘퓨어스템 오에이 키트주’ 임상 순항… “기술수출에 최선”",IT/과학
2,"DN솔루션즈, 獨 테크니컬 센터 열어…""유럽 시장 잡는다""",IT/과학
3,대구경북과학기술원 신임 총장에 이건우 서울대 전 공대 학장,IT/과학
4,울산 남구 테크노산업로 일원 자율주행차 시범운행지구 지정,IT/과학
5,"유니스트, ‘세계에서 가장 영향력 있는 연구자’ 9명",IT/과학
6,"이용훈 총장, 과기부 장관 ‘물망’…유니스트는?",IT/과학
7,"쿠쿠전자, 대용량 '아이편한 가습기 7S' 출시",IT/과학
8,욕설 논란에 카르텔 폭로까지…스텝 꼬인 김범수 쇄신안,IT/과학
9,“이런 XXX같은” 욕설 회의... 카카오 임원 “문제점 지적하다 실수”,IT/과학


## 데이터 수집 및 전처리

### 데이터 수집

In [12]:
# 다른 카테고리의 뉴스들도 수집
code_list = [100, 101, 102, 103, 104, 105]
code_list

[100, 101, 102, 103, 104, 105]

#### make_total_data

특정 날짜의 여러 카테고리의 뉴스를 수집하는 함수 </br>
코드리스트, 페이수, 날짜를 입력 받는다.

In [13]:
def make_total_data(page_num, code_list, date):
    df = None
    
    for code in code_list:
        url_list = make_urllist(page_num=page_num, code=code, date=date)
        df_temp = make_data(url_list, code)
        print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')

        if df is not None:
            df = pd.concat([df, df_temp])
        else:
            df = df_temp

    return df
            

In [14]:
df = make_total_data(1, code_list, 20231128)

100번 코드에 대한 데이터를 만들었습니다.
101번 코드에 대한 데이터를 만들었습니다.
102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.
104번 코드에 대한 데이터를 만들었습니다.
105번 코드에 대한 데이터를 만들었습니다.


In [15]:
print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  120


In [16]:
df.sample(10)

,news,code
16,USA WHITE HOUSE BIDEN CARTER,세계
9,“이런 XXX같은” 욕설 회의... 카카오 임원 “문제점 지적하다 실수”,IT/과학
5,尹 “선의 기대 평화는 허상에 불과”,정치
18,운동 효과 확 높이려면… 끝나고 ‘이것’ 꼭 해야,생활/문화
14,지하 1.5km 정밀 탐사…‘땅속 내시경’ K-DEV 개발,IT/과학
12,"농협은행, ELS 판매 중단…홍콩H지수 폭락에 손실 우려",경제
18,최종 PT하는 최태원 대한상의 회장,정치
19,"건강 이상설 휩싸인 프란치스코 교황, 내달 두바이 기후협약 회의 참석",세계
14,금고 아래 손 넣으니 돈다발…고소득 유튜버도 고액 체납자 포함,경제
2,이스라엘-하마스 서로 교전중단 합의 위반 주장,세계


#### 대량 크롤링

In [17]:
df = make_total_data(50, code_list, 20231128)

100번 코드에 대한 데이터를 만들었습니다.
101번 코드에 대한 데이터를 만들었습니다.
102번 코드에 대한 데이터를 만들었습니다.
103번 코드에 대한 데이터를 만들었습니다.
104번 코드에 대한 데이터를 만들었습니다.
105번 코드에 대한 데이터를 만들었습니다.


#### csv file 로 저장

In [18]:
csv_path = '/Users/project/aiffel/news_data.csv'
df.to_csv(csv_path, index=False)

### 데이터 전처리

#### 데이터 정제

In [ ]:
news = pd.read_csv(csv_path, sep=',')
df.head()

In [ ]:
# 정규 표현식을 이용해서 한글 외의 문자는 전부 제거
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

In [ ]:
print(df.isnull().sum())

In [ ]:
# 중복된 샘플들을 제거
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

#### 데이터 탐색

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "NanumGothic"

df['code'].value_counts().plot(kind = 'bar')

In [ ]:
print(df.groupby('code').size().reset_index(name = 'count'))

#### 토근화/불용어 제거

In [ ]:
from konlpy.tag import Mecab
tokenizer = Mecab()

stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단',
             '따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [ ]:
text_data = preprocessing(df['news'])
print(text_data[0])

## 모델

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

### 데이터 분리

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(text_data, 
                                                    df['code'], 
                                                    random_state=0)

In [ ]:
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

### 각 뉴스 문서를 TF-IDF 벡터로 바꾸고, 이를 통해 나이브 베이즈 분류기를 학습

In [ ]:
# 카운트벡터라이저: 단어의 수를 카운트
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

# 카운트벡터라이저의 결과로부터 TF-IDF 결과 얻기
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# 나이브 베이즈 분류기를 수행 (X_train : TF-IDF 벡터, y_train: 레이블)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

#### 테스트

In [19]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [20]:
new_sent = preprocessing(["민주당 일각에서 법사위의 체계·자구 심사 기능을 없애야 한다는 \
                           주장이 나오는 데 대해 “체계·자구 심사가 법안 지연의 수단으로 \
                          쓰이는 것은 바람직하지 않다”면서도 “국회를 통과하는 법안 중 위헌\
                          법률이 1년에 10건 넘게 나온다. 그런데 체계·자구 심사까지 없애면 매우 위험하다”고 반박했다."])
print(clf.predict(tfidf_vectorizer(new_sent)))

NameError: name 'preprocessing' is not defined

In [ ]:
new_sent = preprocessing(["인도 로맨틱 코미디 영화 <까립까립 싱글>(2017)을 봤을 때 나는 두 눈을 의심했다. \
                          저 사람이 남자 주인공이라고? 노안에 가까운 이목구비와 기름때로 뭉친 파마머리와, \
                          대충 툭툭 던지는 말투 등 전혀 로맨틱하지 않은 외모였다. 반감이 일면서 \
                          ‘난 외모지상주의자가 아니다’라고 자부했던 나에 대해 회의가 들었다.\
                           티브이를 꺼버릴까? 다른 걸 볼까? 그런데, 이상하다. 왜 이렇게 매력 있지? 개구리와\
                            같이 툭 불거진 눈망울 안에는 어떤 인도 배우에게서도 느끼지 못한 \
                            부드러움과 선량함, 무엇보다 슬픔이 있었다. 2시간 뒤 영화가 끝나고 나는 완전히 이 배우에게 빠졌다"])
print(clf.predict(tfidf_vectorizer(new_sent)))

In [ ]:
new_sent = preprocessing(["20분기 연속으로 적자에 시달리는 LG전자가 브랜드 이름부터 성능, 디자인까지 대대적인 변화를 \
                          적용한 LG 벨벳은 등장 전부터 온라인 커뮤니티를 뜨겁게 달궜다. 사용자들은 “디자인이 예쁘다”, \
                          “슬림하다”는 반응을 보이며 LG 벨벳에 대한 기대감을 드러냈다."])
print(clf.predict(tfidf_vectorizer(new_sent)))

### 성능 확인

In [ ]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(classification_report(y_test, y_pred))

## 모델 성능 개선하기

### 형태소 분석기 변경

In [1]:
from konlpy.tag import Kkma


### 불용어 추가

### 다른 날짜 데이터 추가